In [839]:
%matplotlib inline
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [840]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(10, 8)})

In [841]:
import sys
sys.path.append('/Users/bakebrain/src')
import pandas as pd
import asyncio
import json

In [842]:
from mina_schema import mina_explorer_schema
from mina_schema import mina_schema

In [843]:
from MinaClient import Client, Currency, CurrencyFormat
from sgqlc.operation import Operation

from datetime import datetime
from dateutil.parser import parse

In [844]:
mina_explorer_endpoint = 'https://graphql.minaexplorer.com/'
mina_explorer_client = Client(endpoint=mina_explorer_endpoint)

In [845]:
foundation_delegations = [
    "B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1jgwZS8BzXS",
    "B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544vGW6FBZ6a1",
    "B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD7beYumWTB",
    "B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3BbBVkpeC1"
] 

In [846]:
public_key = "B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h1o7pvLPAN" #mina explorer
ledger_hash = "jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4thw2Ee"

op = Operation(mina_explorer_schema.Query)

query = mina_explorer_schema.StakeQueryInput(delegate=public_key, ledger_hash=ledger_hash)

#stakes = op.stakes(query={'delegate': public_key, 'ledger_hash': ledger_hash}, limit=1000)
stakes = op.stakes(query=query, limit=1000)
res = mina_explorer_client.send_any_query(op)

using: https://graphql.minaexplorer.com/


In [847]:
stakes_df = pd.json_normalize(res['data']['stakes'], sep='_')

In [848]:
stakes_df.shape

(110, 28)

In [849]:
stakes_df['is_foundation_delegation'] = stakes_df.public_key.isin(foundation_delegations)

In [850]:
stakes_df.timing_timed_weighting.fillna(1, inplace=True)

In [851]:
total_staking_balance_foundation = stakes_df[stakes_df.is_foundation_delegation].balance.sum()
total_balance = stakes_df.balance.sum()

In [852]:
print(f"The pool total staking balance is: {total_balance}")
print(f"The Foundation delegation balance is: {total_staking_balance_foundation}")
print(f"There are {stakes_df.shape[0]} delegates in the pool")

The pool total staking balance is: 10883171.794181166
The Foundation delegation balance is: 3705636.6277516503
There are 110 delegates in the pool


In [853]:
stakes_df.columns

Index(['balance', 'chainId', 'delegate', 'epoch', 'ledgerHash', 'nonce',
       'public_key', 'receipt_chain_hash', 'token', 'voting_for',
       'delegationTotals_countDelegates', 'delegationTotals_totalDelegated',
       'permissions_edit_state', 'permissions_send',
       'permissions_set_delegate', 'permissions_set_permissions',
       'permissions_set_verification_key', 'permissions_stake',
       'timing_cliff_amount', 'timing_cliff_time',
       'timing_initial_minimum_balance', 'timing_timed_epoch_end',
       'timing_timed_in_epoch', 'timing_timed_weighting',
       'timing_untimed_slot', 'timing_vesting_increment',
       'timing_vesting_period', 'timing', 'is_foundation_delegation'],
      dtype='object')

In [854]:
stakes_df[['public_key', 'balance', 'timing_timed_weighting', 'is_foundation_delegation', 'epoch']].head(n=10)

,public_key,balance,timing_timed_weighting,is_foundation_delegation,epoch
0,B62qkbdgRRJJfqcyVd23s9tgCkNYuGMCmZHKijnJGqYgs9...,6697.0,0.0,False,1
1,B62qqMo7X8i2NnMxrtKf3PAWfbEADk4V1ojWhGeH6Gvye9...,32558.0,0.0,False,1
2,B62qkrhfb1e3fV2HCsFxvnjKp1Yu4UyVpytucWDW16Ri3r...,7000.0,0.0,False,1
3,B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F8...,19200.0,0.0,False,1
4,B62qqrn3yzWRDJrUni6cRva4t51AcnY1o1pM4xpB78MfHU...,65328.0,0.0,False,1
5,B62qoB8RRURcit5keJXvq7uXzYkgN4Lsz5GFaVpGYdA9vA...,66000.0,0.0,False,1
6,B62qowpMhZ2Ww7b8xQxcK7rrpfsL5Nt5Yz5uxaizUBKqpe...,66000.0,0.0,False,1
7,B62qrdiTDeX3AP6aHn62WUsQ3dT7mH7zA6YUGmJ5R9FJDT...,66000.0,0.0,False,1
8,B62qrJb5c4yaeL5fDCrEU5tGsmJWbcfnkdW1pMQbGT1rAn...,66000.0,0.0,False,1
9,B62qrR8VjjKrijdZ9HgUg5D33CrNCWhDdJr8gvmdFFCizC...,66000.0,0.0,False,1


In [855]:
stakes_df.epoch.value_counts()

1    110
Name: epoch, dtype: int64

In [856]:
#The pool total staking balance is: 10883171.794181166
#The Foundation delegation balance is: 3705636.6277516503

# get blocks

In [857]:
epoch = 0
block_height_gte = 0
block_height_lte = 5910

consensus_state = mina_explorer_schema.BlockProtocolStateConsensusStateQueryInput(
    epoch=epoch
)
protocol_state = mina_explorer_schema.BlockProtocolStateQueryInput(
    consensus_state=consensus_state
)

op = Operation(mina_explorer_schema.Query)

query = mina_explorer_schema.BlockQueryInput(
    block_height_gte=block_height_gte,
    block_height_lte=block_height_lte,
    creator=public_key,
    canonical=True,
    protocol_state=protocol_state,
)

blocks = op.blocks(
    query=query,
    limit=500,
)

blocks.transactions()
blocks.block_height()
blocks.tx_fees()
blocks.snark_fees()
blocks.block_height()
blocks.state_hash()

res = mina_explorer_client.send_any_query(op)

using: https://graphql.minaexplorer.com/


In [858]:
#op.__to_graphql__(auto_select_depth=3)

In [859]:
df = pd.json_normalize(res['data']['blocks'], sep='_')

In [860]:
df.head()

,blockHeight,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_feeTransfer,transactions_userCommands
0,5044,0,3NKwM2qNH3mHJUnVW7ceDD1vZmRwu6JRxh3yS9KURv84LS...,30000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'fee': '30000000', 'recipient': 'B62qk9WYHu2...","[{'amount': 1000, 'blockHeight': 5044, 'blockS..."
1,4933,0,3NLsUwqgg9wLgs2jPt4fc7C6nCFVcwV2fEtUvBgayJ1Pj8...,20000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'fee': '20000000', 'recipient': 'B62qk9WYHu2...","[{'amount': 1000, 'blockHeight': 4933, 'blockS..."
2,4863,0,3NLxWu1srPkcm5oaSDg6Bv6rTfeRiDToZHRs36AfP5weXQ...,31000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'fee': '31000000', 'recipient': 'B62qk9WYHu2...","[{'amount': 0, 'blockHeight': 4863, 'blockStat..."
3,4666,0,3NKAPj9zPTvy17i7uQ9ePGp9U377A4divJf2Qiu5dr2SQ3...,40000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'fee': '40000000', 'recipient': 'B62qk9WYHu2...","[{'amount': 1000, 'blockHeight': 4666, 'blockS..."
4,4594,0,3NKu4jk4AaCF1V9JJDGjMsPpAHYpUX37YosHSg6pLBrGjt...,20000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'fee': '20000000', 'recipient': 'B62qk9WYHu2...","[{'amount': 1000, 'blockHeight': 4594, 'blockS..."


In [861]:
for pos,row in df.head().iterrows():
    print(row['blockHeight'])
    

5044
4933
4863
4666
4594


In [862]:
print(f"We won these blocks: {df.blockHeight.tolist()}")
#print(f"We are paying out {all_blocks_total_rewards} nanomina in this window.")

We won these blocks: [5044, 4933, 4863, 4666, 4594, 4517, 4492, 4473, 4417, 4406, 4232, 4160, 4136, 4037, 3953, 3890, 3861, 3795, 3729, 3690, 3657, 3642, 3540, 3517, 3511, 3425, 3423, 3358, 3346, 3298, 3287, 3263, 3095, 3081, 3035, 3013, 3001, 2938, 2903, 2863, 2773, 2721, 2694, 2679, 2618, 2367, 2357, 2317, 2293, 2240, 2211, 2194, 2152, 2107, 2073, 2059, 1946, 1900, 1879, 1873, 1862, 1808, 1793, 1779, 1772, 1550, 1547, 1471, 1368, 1242, 1067, 961, 906, 826, 805, 629, 621, 464, 424, 409, 345, 265, 210, 174, 139, 39]


In [863]:
##### works till here

In [864]:
df.shape

(86, 8)

In [865]:
tmp_df = df.explode(column="transactions_feeTransfer")

fees_df = pd.concat(
    [
        tmp_df.drop(["transactions_feeTransfer"], axis=1),
        tmp_df.transactions_feeTransfer.apply(pd.Series),
    ],
    axis=1,
)

fees_df.fee.fillna(0, inplace=True)
fees_df.fee = fees_df.fee.astype(float)

In [866]:
#fees_df.blockHeight.value_counts().values

In [867]:
fees_df.head()

,blockHeight,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_userCommands,0,fee,recipient,type
0,5044,0,3NKwM2qNH3mHJUnVW7ceDD1vZmRwu6JRxh3yS9KURv84LS...,30000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'amount': 1000, 'blockHeight': 5044, 'blockS...",NaN,30000000.0,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,Fee_transfer
1,4933,0,3NLsUwqgg9wLgs2jPt4fc7C6nCFVcwV2fEtUvBgayJ1Pj8...,20000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'amount': 1000, 'blockHeight': 4933, 'blockS...",NaN,20000000.0,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,Fee_transfer
2,4863,0,3NLxWu1srPkcm5oaSDg6Bv6rTfeRiDToZHRs36AfP5weXQ...,31000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'amount': 0, 'blockHeight': 4863, 'blockStat...",NaN,31000000.0,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,Fee_transfer
3,4666,0,3NKAPj9zPTvy17i7uQ9ePGp9U377A4divJf2Qiu5dr2SQ3...,40000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'amount': 1000, 'blockHeight': 4666, 'blockS...",NaN,40000000.0,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,Fee_transfer
4,4594,0,3NKu4jk4AaCF1V9JJDGjMsPpAHYpUX37YosHSg6pLBrGjt...,20000000,720000000000,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,"[{'amount': 1000, 'blockHeight': 4594, 'blockS...",NaN,20000000.0,B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTn...,Fee_transfer


In [868]:
tmp_df = pd.DataFrame(index=df.blockHeight)

In [869]:
##### exp

In [870]:
tmp_df = pd.DataFrame(index=df.blockHeight)

coinbase_dict = dict(zip(fees_df.blockHeight, fees_df.transactions_coinbase))

tmp_df['coinbase'] = tmp_df.index.map(coinbase_dict).astype(int)

fee_transfer_df = (
    fees_df[fees_df["type"] == "Fee_transfer"]
    .groupby("blockHeight")
    .agg(fee_transfer=("fee", "sum"))
) 

tmp_df = tmp_df.join(fee_transfer_df)

fee_transfer_via_coinbase_df = (
    fees_df[fees_df["type"] == "Fee_transfer_via_coinbase"]
    .groupby("blockHeight")
    .agg(fee_transfer_via_coinbase=("fee", "sum"))
)

tmp_df = tmp_df.join(fee_transfer_via_coinbase_df)

fee_transfer_to_creator_df = fees_df[
    (fees_df["type"] == "Fee_transfer")
    & (fees_df.recipient == fees_df.transactions_coinbaseReceiverAccount_publicKey)
].groupby("blockHeight").agg(fee_transfer_to_creator=("fee", "sum"))

tmp_df = tmp_df.join(fee_transfer_to_creator_df)

tmp_df.fillna(0, inplace=True)

tmp_df["fee_transfer_to_snarkers"] = (
    tmp_df.fee_transfer - tmp_df.fee_transfer_to_creator
)

tmp_df['supercharged_weighting'] = 1 + (
    1
    / (
        1
        + (tmp_df.fee_transfer_to_creator - tmp_df.fee_transfer_to_snarkers)
        / (tmp_df.coinbase - tmp_df.fee_transfer_via_coinbase)
    )
)

tmp_df["total_rewards"] = (
    tmp_df.coinbase + tmp_df.fee_transfer_to_creator - tmp_df.fee_transfer_via_coinbase
)

tmp_df['total_fees'] = 0.05 * tmp_df.total_rewards



In [871]:
tmp_df.columns

Index(['coinbase', 'fee_transfer', 'fee_transfer_via_coinbase',
       'fee_transfer_to_creator', 'fee_transfer_to_snarkers',
       'supercharged_weighting', 'total_rewards', 'total_fees'],
      dtype='object')

In [872]:
tmp_df.total_rewards.sum() / 1e9

61921.462

In [873]:
tmp_df.total_fees.sum() / 1e9

3096.0731

In [874]:
stakes_df.columns

Index(['balance', 'chainId', 'delegate', 'epoch', 'ledgerHash', 'nonce',
       'public_key', 'receipt_chain_hash', 'token', 'voting_for',
       'delegationTotals_countDelegates', 'delegationTotals_totalDelegated',
       'permissions_edit_state', 'permissions_send',
       'permissions_set_delegate', 'permissions_set_permissions',
       'permissions_set_verification_key', 'permissions_stake',
       'timing_cliff_amount', 'timing_cliff_time',
       'timing_initial_minimum_balance', 'timing_timed_epoch_end',
       'timing_timed_in_epoch', 'timing_timed_weighting',
       'timing_untimed_slot', 'timing_vesting_increment',
       'timing_vesting_period', 'timing', 'is_foundation_delegation'],
      dtype='object')

In [875]:
tmp_stakes_df = stakes_df[['public_key', 'balance', 'timing_timed_weighting', 'is_foundation_delegation']]

In [876]:
tmp_stakes_df

,public_key,balance,timing_timed_weighting,is_foundation_delegation
0,B62qkbdgRRJJfqcyVd23s9tgCkNYuGMCmZHKijnJGqYgs9...,6.697000e+03,0.0,False
1,B62qqMo7X8i2NnMxrtKf3PAWfbEADk4V1ojWhGeH6Gvye9...,3.255800e+04,0.0,False
2,B62qkrhfb1e3fV2HCsFxvnjKp1Yu4UyVpytucWDW16Ri3r...,7.000000e+03,0.0,False
3,B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F8...,1.920000e+04,0.0,False
4,B62qqrn3yzWRDJrUni6cRva4t51AcnY1o1pM4xpB78MfHU...,6.532800e+04,0.0,False
...,...,...,...,...
105,B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3...,5.027778e+05,0.0,True
106,B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544v...,9.009735e+05,0.0,True
107,B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD...,6.079048e+05,0.0,True
108,B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1j...,1.693981e+06,0.0,True


In [877]:
print(f"The pool total staking balance is: {total_balance}")
print(f"The Foundation delegation balance is: {total_staking_balance_foundation}")
print(f"There are {stakes_df.shape[0]} delegates in the pool")

The pool total staking balance is: 10883171.794181166
The Foundation delegation balance is: 3705636.6277516503
There are 110 delegates in the pool


In [878]:
tmp_df

,coinbase,fee_transfer,fee_transfer_via_coinbase,fee_transfer_to_creator,fee_transfer_to_snarkers,supercharged_weighting,total_rewards,total_fees
blockHeight,,,,,,,,
5044,720000000000,30000000.0,0.0,30000000.0,0.0,1.999958,7.200300e+11,3.600150e+10
4933,720000000000,20000000.0,0.0,20000000.0,0.0,1.999972,7.200200e+11,3.600100e+10
4863,720000000000,31000000.0,0.0,31000000.0,0.0,1.999957,7.200310e+11,3.600155e+10
4666,720000000000,40000000.0,0.0,40000000.0,0.0,1.999944,7.200400e+11,3.600200e+10
4594,720000000000,20000000.0,0.0,20000000.0,0.0,1.999972,7.200200e+11,3.600100e+10
...,...,...,...,...,...,...,...,...
265,720000000000,30000000.0,0.0,30000000.0,0.0,1.999958,7.200300e+11,3.600150e+10
210,720000000000,20000000.0,0.0,20000000.0,0.0,1.999972,7.200200e+11,3.600100e+10
174,720000000000,30000000.0,0.0,30000000.0,0.0,1.999958,7.200300e+11,3.600150e+10


In [964]:
block_height_to_total_rewards_dict = dict(zip(tmp_df.index, tmp_df.total_rewards))

In [965]:
#tmp_stakes_df

In [966]:
tmp_tmp_df = tmp_df.reset_index()[
    ["blockHeight", "coinbase", "supercharged_weighting", "total_rewards", "total_fees"]
]

In [967]:
yolo_df = tmp_tmp_df.merge(tmp_stakes_df, how='cross')
yolo_df

,blockHeight,coinbase,supercharged_weighting,total_rewards,total_fees,public_key,balance,timing_timed_weighting,is_foundation_delegation
0,5044,720000000000,1.999958,7.200300e+11,3.600150e+10,B62qkbdgRRJJfqcyVd23s9tgCkNYuGMCmZHKijnJGqYgs9...,6.697000e+03,0.0,False
1,5044,720000000000,1.999958,7.200300e+11,3.600150e+10,B62qqMo7X8i2NnMxrtKf3PAWfbEADk4V1ojWhGeH6Gvye9...,3.255800e+04,0.0,False
2,5044,720000000000,1.999958,7.200300e+11,3.600150e+10,B62qkrhfb1e3fV2HCsFxvnjKp1Yu4UyVpytucWDW16Ri3r...,7.000000e+03,0.0,False
3,5044,720000000000,1.999958,7.200300e+11,3.600150e+10,B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F8...,1.920000e+04,0.0,False
4,5044,720000000000,1.999958,7.200300e+11,3.600150e+10,B62qqrn3yzWRDJrUni6cRva4t51AcnY1o1pM4xpB78MfHU...,6.532800e+04,0.0,False
...,...,...,...,...,...,...,...,...,...
9455,39,720000000000,1.999986,7.200100e+11,3.600050e+10,B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3...,5.027778e+05,0.0,True
9456,39,720000000000,1.999986,7.200100e+11,3.600050e+10,B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544v...,9.009735e+05,0.0,True
9457,39,720000000000,1.999986,7.200100e+11,3.600050e+10,B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD...,6.079048e+05,0.0,True
9458,39,720000000000,1.999986,7.200100e+11,3.600050e+10,B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1j...,1.693981e+06,0.0,True


In [968]:
grouped = yolo_df[yolo_df.is_foundation_delegation == True].groupby(['blockHeight', 'public_key'])

pay_data_tmp = []
for name, g in grouped:
    block_height = name[0]
    public_key = name[1]    
    f = math.floor((g["balance"].item() / total_balance) * g.coinbase.item() *(1 - 0.05))
    #print(f)
    pay_data_tmp.append({
        'block_height': block_height,
        'public_key': public_key,
        'balance': g.balance.item(),
        'coinbase': g.coinbase.item(),
        'f': f
    })
    
halo_df = pd.DataFrame(pay_data_tmp)

In [969]:
halo_df.groupby('public_key')['f'].sum()

public_key
B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3BbBVkpeC1    2717534960866
B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1jgwZS8BzXS    9156036394420
B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544vGW6FBZ6a1    4869799366150
B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD7beYumWTB    3285750669954
Name: f, dtype: int64

In [974]:
foundation_block_to_total_rewards_dict = halo_df.groupby(['block_height'])['f'].sum().to_dict()

In [985]:
foundation_block_to_total_rewards_dict

{39: 232896760365,
 139: 232896760365,
 174: 232896760365,
 210: 232896760365,
 265: 232896760365,
 345: 232896760365,
 409: 232896760365,
 424: 232896760365,
 464: 232896760365,
 621: 232896760365,
 629: 232896760365,
 805: 232896760365,
 826: 232896760365,
 906: 232896760365,
 961: 232896760365,
 1067: 232896760365,
 1242: 232896760365,
 1368: 232896760365,
 1471: 232896760365,
 1547: 232896760365,
 1550: 232896760365,
 1772: 232896760365,
 1779: 232896760365,
 1793: 232896760365,
 1808: 232896760365,
 1862: 232896760365,
 1873: 232896760365,
 1879: 232896760365,
 1900: 232896760365,
 1946: 232896760365,
 2059: 232896760365,
 2073: 232896760365,
 2107: 232896760365,
 2152: 232896760365,
 2194: 232896760365,
 2211: 232896760365,
 2240: 232896760365,
 2293: 232896760365,
 2317: 232896760365,
 2357: 232896760365,
 2367: 232896760365,
 2618: 232896760365,
 2679: 232896760365,
 2694: 232896760365,
 2721: 232896760365,
 2773: 232896760365,
 2863: 232896760365,
 2903: 232896760365,
 2938: 2

In [991]:
yolo_df[yolo_df.is_foundation_delegation == False].shape

(9116, 9)

In [992]:

grouped_no_f = yolo_df[yolo_df.is_foundation_delegation == False].groupby(['blockHeight', 'public_key'])
c = []
pay_data_tmp_no_f = []
for name, g in grouped_no_f:
    block_height = name[0]
    public_key = name[1] 
    supercharged_contribution = (
                (g['supercharged_weighting'].item() - 1) * g["timing_timed_weighting"].item()) + 1
    effective_stake = g["balance"].item() * supercharged_contribution
    pay_data_tmp_no_f.append({
        'block_height': block_height,
        'public_key':public_key,
        'effective_stake':effective_stake
    })
pay_data_tmp_no_f_df = pd.DataFrame(pay_data_tmp_no_f)

block_to_stake_dict = pay_data_tmp_no_f_df.groupby(['block_height'])['effective_stake'].sum().to_dict()

eff_stake_df = pay_data_tmp_no_f_df.groupby(['block_height', 'public_key'])['effective_stake'].sum().to_frame()
eff_stake_df

effective_stake
block_height public_key                                                         
39           B62qiWSe63weqBN3vpnUyagnMkYmvRwKKQJ4dKckuJFsojU...     66000.000000
             B62qiX7wCtUbpzgJavPKbfBcLJ6nyYB88cWjfwjWZiyZZLS...    203053.166619
             B62qid7mcNh29WVifR2qWQDLJ9dRu94EUcbsgrTr9fdFMGs...     66000.000000
             B62qieizHWA2zusWh8y9eCzqA6dvgGNSs5RJGHPmAGcBtdg...     66000.000000
             B62qikygSmv245hJZQzAhAqxfqeNiyPDZUS9MME4HkP9oFN...     66000.000000
...                                                                          ...
5044         B62qrYkGp44a78p3t6uifiBq2wwctJw3k8u88sKLLAgdAZF...     66000.000000
             B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F85...     19200.000000
             B62qrdiTDeX3AP6aHn62WUsQ3dT7mH7zA6YUGmJ5R9FJDTa...     66000.000000
             B62qreVP6TddxX5CySuko7gqxVwzRmQyYzraHp85dzYG9g9...     66000.000000
             B62qrzTgzDJ6n1UojoypnHx4P1onR61r3RZkFaqFcgQ1mVU...     66000.000000

[9116 rows x 1 columns]

In [999]:
pay_data_tmp_no_f_df.head(n=10)

,block_height,public_key,effective_stake
0,39,B62qiWSe63weqBN3vpnUyagnMkYmvRwKKQJ4dKckuJFsoj...,66000.000000
1,39,B62qiX7wCtUbpzgJavPKbfBcLJ6nyYB88cWjfwjWZiyZZL...,203053.166619
2,39,B62qid7mcNh29WVifR2qWQDLJ9dRu94EUcbsgrTr9fdFMG...,66000.000000
3,39,B62qieizHWA2zusWh8y9eCzqA6dvgGNSs5RJGHPmAGcBtd...,66000.000000
4,39,B62qikygSmv245hJZQzAhAqxfqeNiyPDZUS9MME4HkP9oF...,66000.000000
5,39,B62qiwepAfGCvyDBMx8acQ2wTwEDLN4ZkAK4VEwtuDqawK...,66000.000000
6,39,B62qixTkfYwADQSS9SHL9NRBoNKvmfCYxQAPhoLiZDA5Rs...,66000.000000
7,39,B62qj49ZC1dvWzGNMd5VVxSVQhZv4rzoKd7TdsVSUAsHZ7...,66000.000000
8,39,B62qj5TbymjFWUsjHnCDNfzbFbacKzXwnHdgDJDoAZwcs5...,66000.000000
9,39,B62qjBp3TgwngdrKkK69NjrhTFS9KSTriLFiv2FKvnhGfj...,66000.000000


In [995]:
eff_stake_df.loc[(39, 'B62qiX7wCtUbpzgJavPKbfBcLJ6nyYB88cWjfwjWZiyZZLS7weCDxwm')].item()

203053.166619445

In [957]:
block_to_stake_dict[39]

14355070.332859034

In [1009]:
shit_df_data = []
for name, g in grouped_no_f:
    block_height = name[0]
    public_key = name[1] 
    
    w = eff_stake_df.loc[(block_height, public_key)].item() / block_to_stake_dict[block_height]
    print(w)
    
    block_total_rewards = block_height_to_total_rewards_dict[block_height]
    
    foundation_payouts = foundation_block_to_total_rewards_dict[block_height]
    
    block_pool_share = block_total_rewards - (foundation_payouts / (1 - 0.05))
    #print(block_pool_share)
    
    block_total = math.floor(block_pool_share *w * (1 - 0.05))
    #print(block_total)
    
    shit_df_data.append({
        'block_height': block_height,
        'public_key': public_key,
        'balance': g.balance.item(),
        'coinbase': g.coinbase.item(),
        'block_total': block_total
    })

0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009330501132649191
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009752651624390673
0.0013932359463415247
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.00027864718926830494
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195

0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009330501132649191
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009752651624390673
0.0013932359463415247
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.00027864718926830494
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.00027864718926830494
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.00027864718926830494
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009

0.009195357245854064
0.00027864718926830494
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.004536097594098737
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009101731790259913
0.009195357245854064
0.0006966179731707624
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.04274767165619376
0.009195

0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009330501132649191
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009752651624390673
0.0013932359463415247
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0002786

0.0
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.004536097594098737
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009101731790259913
0.009195357245854064
0.0006966179731707624
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195

...

0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.004536097594098737
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009101731790259913
0.009195357245854064
0.0006966179731707624
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195

0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.004536097594098737
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009101731790259913
0.009195357245854064
0.0006966179731707624
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195

0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009101731790259913
0.009195357245854064
0.0006966179731707624
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009330501132649191
0.009195357245854064
0.009195357245854064
0.009195357

0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.004536097594098737
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009101731790259913
0.009195357245854064
0.0006966179731707624
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0091953572

0.04274767165619376
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0026750130169757276
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.028290097075268576
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009330501132649191
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.0009752651624390673
0.0013932359463415247
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.009195357245854064
0.00919535

In [1003]:
tmp_test_df = pd.DataFrame(shit_df_data)

In [1001]:
pd.DataFrame(shit_df_data).groupby('public_key')['block_total'].sum() / 1e9

public_key
B62qiWSe63weqBN3vpnUyagnMkYmvRwKKQJ4dKckuJFsojUAYY56mfx     356.745540
B62qiX7wCtUbpzgJavPKbfBcLJ6nyYB88cWjfwjWZiyZZLS7weCDxwm    1097.550174
B62qid7mcNh29WVifR2qWQDLJ9dRu94EUcbsgrTr9fdFMGsPQzWv8Qo     356.745540
B62qieizHWA2zusWh8y9eCzqA6dvgGNSs5RJGHPmAGcBtdgr8BRSfEg     356.745540
B62qikygSmv245hJZQzAhAqxfqeNiyPDZUS9MME4HkP9oFNhij85XUp     356.745540
                                                              ...     
B62qrYkGp44a78p3t6uifiBq2wwctJw3k8u88sKLLAgdAZFK1G8UGcH     356.745540
B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F85T4DmtjC7     103.780521
B62qrdiTDeX3AP6aHn62WUsQ3dT7mH7zA6YUGmJ5R9FJDTac4j6DmPA     356.745540
B62qreVP6TddxX5CySuko7gqxVwzRmQyYzraHp85dzYG9g9ixPbVL3D     356.745540
B62qrzTgzDJ6n1UojoypnHx4P1onR61r3RZkFaqFcgQ1mVUPY6cJzSr     356.745540
Name: block_total, Length: 106, dtype: float64

In [934]:

    #else:
    #    supercharged_contribution = (
    #            (g['supercharged_weighting'].item() - 1) * g["timing_timed_weighting"].item()) + 1
    #    effective_stake = g["balance"].item() * supercharged_contribution

In [958]:
halo_df

,block_height,public_key,balance,coinbase,f
0,39,B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3...,5.027778e+05,720000000000,31599243731
1,39,B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1j...,1.693981e+06,720000000000,106465539470
2,39,B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544v...,9.009735e+05,720000000000,56625574025
3,39,B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD...,6.079048e+05,720000000000,38206403139
4,139,B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3...,5.027778e+05,720000000000,31599243731
...,...,...,...,...,...
339,4933,B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD...,6.079048e+05,720000000000,38206403139
340,5044,B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3...,5.027778e+05,720000000000,31599243731
341,5044,B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1j...,1.693981e+06,720000000000,106465539470
342,5044,B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544v...,9.009735e+05,720000000000,56625574025


public_key
B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3BbBVkpeC1    2717534960866
B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1jgwZS8BzXS    9156036394420
B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544vGW6FBZ6a1    4869799366150
B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD7beYumWTB    3285750669954
Name: f, dtype: int64

In [900]:
| B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3BbBVkpeC1 | 502777.775000000  |  2717534960866  | 2717.534960866 |    True    |
| B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544vGW6FBZ6a1 | 900973.465000000  |  4869799366150  | 4869.799366150 |    True    |
| B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD7beYumWTB | 607904.750000000  |  3285750669954  | 3285.750669954 |    True    |
| B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1jgwZS8BzXS | 1693980.637751650 |  9156036394420  | 9156.036394420 |    True    |

344

In [888]:
h / 1e9

20029.12139139

20029121391390

In [889]:
232896760365 / 1e9

232.896760365

In [893]:
tmp_df.shape

(86, 8)

In [894]:
4 * 86

344

In [783]:
def get_total(row):
    if row['is_foundation_delegation']:
        return (p["staking_balance"] / total_staking_balance) * coinbase * (1 - fee)
        
        
        
tmp_stakes_df.tail().apply(lambda row: get_total(row), axis=1)
#tmp_stakes_df['total'] = tmp_stakes_df.apply(lambda row: get_total(row), axis=1)

yolo
yolo
yolo
yolo


105    None
106    None
107    None
108    None
109    None
dtype: object

In [771]:
print(f"The pool total staking balance is: {total_balance}")
print(f"The Foundation delegation balance is: {total_staking_balance_foundation}")
print(f"There are {stakes_df.shape[0]} delegates in the pool")

Index(['balance', 'chainId', 'delegate', 'epoch', 'ledgerHash', 'nonce',
       'public_key', 'receipt_chain_hash', 'token', 'voting_for',
       'delegationTotals_countDelegates', 'delegationTotals_totalDelegated',
       'permissions_edit_state', 'permissions_send',
       'permissions_set_delegate', 'permissions_set_permissions',
       'permissions_set_verification_key', 'permissions_stake',
       'timing_cliff_amount', 'timing_cliff_time',
       'timing_initial_minimum_balance', 'timing_timed_epoch_end',
       'timing_timed_in_epoch', 'timing_timed_weighting',
       'timing_untimed_slot', 'timing_vesting_increment',
       'timing_vesting_period', 'timing', 'is_foundation_delegation'],
      dtype='object')

In [ ]:
##### end exp

In [709]:
fee_transfer_df = (
    fees_df[fees_df["type"] == "Fee_transfer"]
    .groupby("blockHeight")
    .agg({"fee": "sum"})
)  

fee_transfer_for_coinbase_df = (
    fees_df[fees_df["type"] == "Fee_transfer_via_coinbase"]
    .groupby("blockHeight")
    .agg({"fee": "sum"})
)

total_fee_transfers_to_creator_df = fees_df[
    (fees_df["type"] == "Fee_transfer")
    & (fees_df.recipient == fees_df.transactions_coinbaseReceiverAccount_publicKey)
].groupby("blockHeight").agg({"fee": "sum"})

#cb_df = fees_df

In [710]:
tmp_df["fee_transfers"] = tmp_df.index.map(fee_transfer_df.to_dict()["fee"])
tmp_df["coinbase_fee_transfers"] = tmp_df.index.map(
    fee_transfer_for_coinbase_df.to_dict()["fee"]
)
tmp_df["fee_transfers_to_creator"] = tmp_df.index.map(
    total_fee_transfers_to_creator_df.to_dict()["fee"]
)

tmp_df.fillna(0, inplace=True)

tmp_df["fee_transfer_to_snarkers"] = (
    tmp_df.fee_transfers - tmp_df.fee_transfers_to_creator
)
tmp_df["coinbase"] = tmp_df.index.map(coinbase_dict).astype(float)

In [711]:
tmp_df['supercharged_weighting'] = 1 + (
    1
    / (
        1
        + (tmp_df.fee_transfers_to_creator - tmp_df.fee_transfer_to_snarkers)
        / (tmp_df.coinbase - tmp_df.coinbase_fee_transfers)
    )
)

In [712]:
#    total_rewards = int(
#        b["transactions"]["coinbase"]
#    ) + total_fee_transfers_to_creator - fee_transfer_for_coinbase

In [713]:
tmp_df["total_rewards"] = (
    tmp_df.coinbase + tmp_df.fee_transfers_to_creator - tmp_df.coinbase_fee_transfers
)

In [714]:
tmp_df['total_fees'] = 0.05 * tmp_df.total_rewards

In [715]:
tmp_df

,fee_transfers,coinbase_fee_transfers,fee_transfers_to_creator,fee_transfer_to_snarkers,coinbase,supercharged_weighting,total_rewards,total_fees
blockHeight,,,,,,,,
5044,30000000.0,0.0,30000000.0,0.0,7.200000e+11,1.999958,7.200300e+11,3.600150e+10
4933,20000000.0,0.0,20000000.0,0.0,7.200000e+11,1.999972,7.200200e+11,3.600100e+10
4863,31000000.0,0.0,31000000.0,0.0,7.200000e+11,1.999957,7.200310e+11,3.600155e+10
4666,40000000.0,0.0,40000000.0,0.0,7.200000e+11,1.999944,7.200400e+11,3.600200e+10
4594,20000000.0,0.0,20000000.0,0.0,7.200000e+11,1.999972,7.200200e+11,3.600100e+10
...,...,...,...,...,...,...,...,...
265,30000000.0,0.0,30000000.0,0.0,7.200000e+11,1.999958,7.200300e+11,3.600150e+10
210,20000000.0,0.0,20000000.0,0.0,7.200000e+11,1.999972,7.200200e+11,3.600100e+10
174,30000000.0,0.0,30000000.0,0.0,7.200000e+11,1.999958,7.200300e+11,3.600150e+10


In [689]:
#tmp_df.total_fees.astype(Currency)

In [690]:
stakes_df

,balance,chainId,delegate,epoch,ledgerHash,nonce,public_key,receipt_chain_hash,token,voting_for,...,timing_cliff_time,timing_initial_minimum_balance,timing_timed_epoch_end,timing_timed_in_epoch,timing_timed_weighting,timing_untimed_slot,timing_vesting_increment,timing_vesting_period,timing,is_foundation_delegation
0,6.697000e+03,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qkbdgRRJJfqcyVd23s9tgCkNYuGMCmZHKijnJGqYgs9...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,86400.0,6697.0,True,True,0.0,86400.0,0.0,1.0,NaN,False
1,3.255800e+04,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qqMo7X8i2NnMxrtKf3PAWfbEADk4V1ojWhGeH6Gvye9...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,86400.0,32558.0,True,True,0.0,86400.0,0.0,1.0,NaN,False
2,7.000000e+03,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qkrhfb1e3fV2HCsFxvnjKp1Yu4UyVpytucWDW16Ri3r...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,86400.0,7000.0,True,True,0.0,86400.0,0.0,1.0,NaN,False
3,1.920000e+04,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qraStik5h6MHyJdB39Qd2gY2pPHaKsZFLWVNEv2h3F8...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,86400.0,19200.0,True,True,0.0,86400.0,0.0,1.0,NaN,False
4,6.532800e+04,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qqrn3yzWRDJrUni6cRva4t51AcnY1o1pM4xpB78MfHU...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,86400.0,65328.0,True,True,0.0,86400.0,0.0,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,5.027778e+05,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qmQAFPta1Q3c7wXHxXRKnE3uWyBYZCLb8frdHEgavi3...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,691200.0,1.0,True,True,0.0,691200.0,0.0,1.0,NaN,True
106,9.009735e+05,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qn71s63yywMUCcFhP4iCata7HpgyrvmGjpKa1D9544v...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,691200.0,1.0,True,True,0.0,691200.0,0.0,1.0,NaN,True
107,6.079048e+05,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qqEV4oP7w2jLQGckvZzdWjfdLKySKHJ3tNU5niRjpPD...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,691200.0,1.0,True,True,0.0,691200.0,0.0,1.0,NaN,True
108,1.693981e+06,5f704c,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,1,jx7buQVWFLsXTtzRgSxbYcT8EYLS8KCZbLrfDcJxMtyy4t...,0,B62qmsYXFNNE565yv7bEMPsPnpRCsMErf7J2v5jMnuKQ1j...,2mzbV7WevxLuchs2dAMY4vQBS6XttnCUF8Hvks4XNBQ5qi...,1,3NK2tkzqqK5spR2sZ7tujjqPksL45M3UUrcA4WhCkeiPtn...,...,345600.0,1.0,True,True,0.0,345600.0,0.0,1.0,NaN,True


In [677]:
#tmp_df.loc[4666].values

In [ ]:
720040000000

In [ ]:
    total_rewards = int(
        b["transactions"]["coinbase"]
    ) + total_fee_transfers_to_creator - fee_transfer_for_coinbase

In [590]:
fees_df['type'].value_counts()

Fee_transfer    74
Name: type, dtype: int64

In [582]:
total_fee_transfers_df = fees_df[fees_df['type'] == 'Fee_transfer'].groupby('blockHeight').agg({'fee': 'sum'})

In [591]:
total_fee_transfers_df

,fee
blockHeight,
39,10000000
139,20000000
174,30000000
210,20000000
265,30000000
...,...
4594,20000000
4666,40000000
4863,31000000


In [589]:
fees_df.groupby(['blockHeight', 'type']).agg({'fee': 'sum'})

,,fee
blockHeight,type,
39,Fee_transfer,10000000
139,Fee_transfer,20000000
174,Fee_transfer,30000000
210,Fee_transfer,20000000
265,Fee_transfer,30000000
...,...,...
4594,Fee_transfer,20000000
4666,Fee_transfer,40000000
4863,Fee_transfer,31000000


In [583]:
total_fee_transfers_df

,fee
blockHeight,
39,10000000
139,20000000
174,30000000
210,20000000
265,30000000
...,...
4594,20000000
4666,40000000
4863,31000000


In [584]:
fee_transfer_for_coinbase_df = fees_df[fees_df['type'] == 'Fee_transfer_via_coinbase'].groupby('blockHeight').agg({'fee': 'sum'})

In [585]:
fee_transfer_for_coinbase_df

,fee
blockHeight,


In [586]:
fees_df[fees_df['type'] == 'Fee_transfer_via_coinbase'].groupby(['blockHeight'])['fee'].transform('count')

Series([], Name: fee, dtype: int64)

In [552]:
total_fee_transfers_to_creator_df = fees_df[
    (fees_df["type"] == "Fee_transfer")
    & (fees_df.recipient == fees_df.transactions_coinbaseReceiverAccount_publicKey)
].groupby("blockHeight").agg({"fee": "sum"})


In [577]:
fee_transfer_to_snarkers_df = total_fee_transfers_df - total_fee_transfers_to_creator_df

In [578]:
fee_transfer_to_snarkers_df

,fee
blockHeight,
39,0
139,0
174,0
210,0
265,0
...,...
4594,0
4666,0
4863,0


In [595]:
supercharged_weighting = 1 + (
        1 / (1 + (
            total_fee_transfers_to_creator_df - fee_transfer_to_snarkers_df
        ) / (0 - 0)))


In [596]:
fee_transfer_for_coinbase_df

,fee
blockHeight,


In [576]:
fee_transfer_for_coinbase_df

,fee
blockHeight,


In [ ]:
supercharged_weighting = 1 + (
    1
    / (
        1
        + (int(total_fee_transfers_to_creator) - int(fee_transfer_to_snarkers))
        / (int(b["transactions"]["coinbase"]) - int(fee_transfer_for_coinbase))
    )
)

In [553]:
    supercharged_weighting = 1 + (
        1 / (1 + (int(total_fee_transfers_to_creator) - int(fee_transfer_to_snarkers)) / (int(b["transactions"]["coinbase"]) - int(fee_transfer_for_coinbase))))

    print(f"supercharged_weighting: {supercharged_weighting}")

,fee
blockHeight,
39,10000000
139,20000000
174,30000000
210,20000000
265,30000000
...,...
4594,20000000
4666,40000000
4863,31000000


In [544]:
fee_transfer_to_creator_df = fees_df[
    (fees_df["type"] == "Fee_transfer")
    & (fees_df.recipient == fees_df.transactions_coinbaseReceiverAccount_publicKey)
].groupby("blockHeight").agg({"fee": "sum"})

fee_transfer_to_creator_df

,fee
blockHeight,
39,10000000
139,20000000
174,30000000
210,20000000
265,30000000
...,...
4594,20000000
4666,40000000
4863,31000000


In [531]:
public_key

'B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h1o7pvLPAN'

B62qk9WYHu2PBYv4EyEubnVQURcwpiV2ysuYYoMdwi8YTnwZQ7H4bLM

In [368]:
user_cmd_df = df.explode(column='transaction<s_userCommands')
test_df = pd.concat([user_cmd_df, user_cmd_df.transactions_userCommands.apply(pd.Series)], axis=1)

In [369]:
test_df.shape

(187, 31)

In [376]:
test_df[test_df['from'] == public_key]

,blockHeight,canonical,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_feeTransfer,transactions_userCommands,0,...,id,isDelegation,kind,memo,nonce,receiver,source,to,toAccount,token


In [375]:
test_df[['canonical', 'from', 'to']]

,canonical,from,to
0,True,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...,B62qqhURJQo3CvWC3WFo9LhUhtcaJWLBcJsaA3DXaU2GH5...
0,True,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...,B62qqhURJQo3CvWC3WFo9LhUhtcaJWLBcJsaA3DXaU2GH5...
0,True,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...,B62qqhURJQo3CvWC3WFo9LhUhtcaJWLBcJsaA3DXaU2GH5...
0,True,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...,B62qqhURJQo3CvWC3WFo9LhUhtcaJWLBcJsaA3DXaU2GH5...
0,True,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...,B62qqhURJQo3CvWC3WFo9LhUhtcaJWLBcJsaA3DXaU2GH5...
...,...,...,...
93,True,B62qre3erTHfzQckNuibViWQGyyKwZseztqrjPZBv6SQF3...,B62qjYanmV7y9njVeH5UHkz3GYBm7xKir1rAnoY4KsEYUG...
93,True,B62qkZuTU66fez9p4qwx2E68zggQ26etV6WDcnSx1vizfa...,B62qpCegcfvogh9pPzYSR32GZR2vnQc3hiF2uZJmVvJL9x...
94,True,B62qre3erTHfzQckNuibViWQGyyKwZseztqrjPZBv6SQF3...,B62qjYanmV7y9njVeH5UHkz3GYBm7xKir1rAnoY4KsEYUG...
94,True,B62qre3erTHfzQckNuibViWQGyyKwZseztqrjPZBv6SQF3...,B62qjYanmV7y9njVeH5UHkz3GYBm7xKir1rAnoY4KsEYUG...


In [359]:
test_df.head(n=1)['source'].apply(lambda x: x['publicKey'])

0    B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...
Name: source, dtype: object

In [336]:
d = test_df[test_df.source.notnull()]['source'].apply(lambda x: x['publicKey'])

In [340]:
tmp_df = pd.DataFrame(d)

In [344]:
tmp_df[tmp_df['source'] != public_key] 

,source
0,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...
0,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...
0,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...
0,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...
0,B62qmt4SqMXTa21iWkxfgy2Dd89HSWTkWpdRzWNrUPLQTC...
...,...
93,B62qre3erTHfzQckNuibViWQGyyKwZseztqrjPZBv6SQF3...
93,B62qkZuTU66fez9p4qwx2E68zggQ26etV6WDcnSx1vizfa...
94,B62qre3erTHfzQckNuibViWQGyyKwZseztqrjPZBv6SQF3...
94,B62qre3erTHfzQckNuibViWQGyyKwZseztqrjPZBv6SQF3...


In [321]:


user_cmd_df = df.explode(column='transactions_userCommands')
test_df = pd.concat([tmp_fees_df, user_cmd_df.transactions_userCommands.apply(pd.Series)], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [178]:
#def get_fees(row, fee_type="Fee_transfer"):
#    print(row)
#    fee_df = pd.DataFrame(row)
#    if fee_df.empty:
#        return 0
#    return fee_df[fee_df['type'] == fee_type].fee.sum()


#df['fee_transfers'] = df.transactions_feeTransfer.apply(lambda row: get_fees(row))

In [205]:
tmp_fees_df = df.explode(column='transactions_feeTransfer')
final_df = pd.concat([tmp_fees_df, tmp_fees_df.transactions_feeTransfer.apply(pd.Series)], axis=1)
#fees_df = tmp_fees_df.transactions_feeTransfer.apply(pd.Series)

In [207]:
final_df.fee.fillna(0, inplace=True)
final_df.fee = final_df.fee.astype(int)

In [208]:
final_df.head()

,blockHeight,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_feeTransfer,transactions_userCommands,0,fee,recipient,type
0,1,0,3NKeMoncuHab5ScarV5ViyF16cJPT4taWNSaTLS64Dp67w...,0,720000000000,B62qiy32p8kAKnny8ZFwoMhYpBppM1DWVCqAPBYNcXnsAH...,NaN,[],NaN,0,NaN,NaN
1,2,0,3NLyWnjZqUECniE1q719CoLmes6WDQAod4vrTeLfN7XXJb...,0,720000000000,B62qqa9g4CFfkSuX2j22S52z6UfcDcS9tMTgQrFKZ21v7G...,NaN,[],NaN,0,NaN,NaN
2,3,0,3NKd5So3VNqGZtRZiWsti4yaEe1fX79yz5TbfG6jBZqgMn...,120000000,720000000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,"{'fee': '120000000', 'recipient': 'B62qqSUUCno...","[{'amount': 10000, 'blockHeight': 3, 'blockSta...",NaN,120000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,Fee_transfer
3,4,0,3NL9qBsNibXPm5Nh8cSg5CCqrbzX5VUVY9gJzAbg7EVCF3...,60000000,720000000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,"{'fee': '60000000', 'recipient': 'B62qjZXMHQij...","[{'amount': 10000, 'blockHeight': 4, 'blockSta...",NaN,60000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,Fee_transfer
4,5,0,3NKQUoBfi9vkbuqtDJmSEYBQrcSo4GjwG8bPCiii4yqM8A...,70000000,720000000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,"{'fee': '70000000', 'recipient': 'B62qkiJuTwdJ...","[{'amount': 1000, 'blockHeight': 5, 'blockStat...",NaN,70000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,Fee_transfer


In [210]:
final_df.groupby('transactions_coinbaseReceiverAccount_publicKey')['fee'].sum().to_frame()

,fee
transactions_coinbaseReceiverAccount_publicKey,
B62qiWcbScnq9W9bhk4oLi53fyH5TL8WxzDG5cqMhbz4SUhz66v7izL,10000000
B62qim7EVe62u3wDsGzAZEMGMaEuwSXYWTCH1WsNvWFgfvz7ysggDxf,20000000
B62qiy32p8kAKnny8ZFwoMhYpBppM1DWVCqAPBYNcXnsAHhnfAAuXgg,0
B62qj3Gzxgb4G4M8CwZRXZPtmVwGJtGfVXVbpMrACNDSqQLoXzSQ9HW,20000000
B62qj8KB2fk59NkV4VuoTkVXHjw8VJzC3ybKrWo7zuDC9xTiWXPygEe,20000000
B62qjA7LFMvKuzFbGZK9yb3wAkBThba1pe5ap8UZx8jEvfAEcnDgDBE,10000000
B62qjBpH4zeWgy4d4p59qUXDA3DtYeCyBrRM286sdXZFw4A8SiswfBh,10000000
B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjCzy3zwSVaq,10000000
B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8TqjZm41ydCVM,80000000


In [206]:
#final_df.fee = final_df.fee.fillna(0)

,blockHeight,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_feeTransfer,transactions_userCommands,0,fee,recipient,type
0,1,0,3NKeMoncuHab5ScarV5ViyF16cJPT4taWNSaTLS64Dp67w...,0,720000000000,B62qiy32p8kAKnny8ZFwoMhYpBppM1DWVCqAPBYNcXnsAH...,NaN,[],NaN,NaN,NaN,NaN
1,2,0,3NLyWnjZqUECniE1q719CoLmes6WDQAod4vrTeLfN7XXJb...,0,720000000000,B62qqa9g4CFfkSuX2j22S52z6UfcDcS9tMTgQrFKZ21v7G...,NaN,[],NaN,NaN,NaN,NaN
2,3,0,3NKd5So3VNqGZtRZiWsti4yaEe1fX79yz5TbfG6jBZqgMn...,120000000,720000000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,"{'fee': '120000000', 'recipient': 'B62qqSUUCno...","[{'amount': 10000, 'blockHeight': 3, 'blockSta...",NaN,120000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,Fee_transfer
3,4,0,3NL9qBsNibXPm5Nh8cSg5CCqrbzX5VUVY9gJzAbg7EVCF3...,60000000,720000000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,"{'fee': '60000000', 'recipient': 'B62qjZXMHQij...","[{'amount': 10000, 'blockHeight': 4, 'blockSta...",NaN,60000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,Fee_transfer
4,5,0,3NKQUoBfi9vkbuqtDJmSEYBQrcSo4GjwG8bPCiii4yqM8A...,70000000,720000000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,"{'fee': '70000000', 'recipient': 'B62qkiJuTwdJ...","[{'amount': 1000, 'blockHeight': 5, 'blockStat...",NaN,70000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,Fee_transfer
...,...,...,...,...,...,...,...,...,...,...,...,...
95,60,0,3NKu5MoEy2xvRMpKCg5FYVJ13paiFPzM3cuof5qr46v7Xf...,10000000,720000000000,B62qoigHEtJCoZ5ekbGHWyr9hYfc6fkZ2A41h9vvVZuvty...,"{'fee': '10000000', 'recipient': 'B62qoigHEtJC...","[{'amount': 1000, 'blockHeight': 60, 'blockSta...",NaN,10000000,B62qoigHEtJCoZ5ekbGHWyr9hYfc6fkZ2A41h9vvVZuvty...,Fee_transfer
96,60,0,3NKCdQt2pAeLqrPYxLHNs7UeusGGrTL8X6HcaFs8yJH4YA...,10000000,720000000000,B62qmvHQzJmT2rKE1F9RemenGRG8BfXT1Kurve3eT4iC2H...,"{'fee': '10000000', 'recipient': 'B62qmvHQzJmT...","[{'amount': 1000, 'blockHeight': 60, 'blockSta...",NaN,10000000,B62qmvHQzJmT2rKE1F9RemenGRG8BfXT1Kurve3eT4iC2H...,Fee_transfer
97,60,0,3NKYioZFVfvLw6BRmfnbpQRKaLcLQzyo6YTZ5Gr2TPtY4R...,10000000,720000000000,B62qmMkbajiY3bdVRjv5bx3yxjp5sBHHQuu8M11sDdjtBy...,"{'fee': '10000000', 'recipient': 'B62qmMkbajiY...","[{'amount': 1000, 'blockHeight': 60, 'blockSta...",NaN,10000000,B62qmMkbajiY3bdVRjv5bx3yxjp5sBHHQuu8M11sDdjtBy...,Fee_transfer
98,61,0,3NL6SMuZKjUrWYLoayQrRmsFbFzwfAp7WZydHkXaGakTzN...,10000000,720000000000,B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjC...,"{'fee': '10000000', 'recipient': 'B62qjBsp2zWg...","[{'amount': 1000, 'blockHeight': 61, 'blockSta...",NaN,10000000,B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjC...,Fee_transfer


In [188]:
#pd.json_normalize(tmp_fees_df.transactions_feeTransfer)

In [ ]:
final_exploded = pd.concat(
    [
        tmp_fees_df.drop(["transactions_feeTransfer"], axis=1),
        tmp_fees_df["transactions_feeTransfer"].apply(pd.Series),
    ],
    axis=1,
)

In [204]:
pd.concat([tmp_fees_df, tmp_fees_df.transactions_feeTransfer.apply(pd.Series)], axis=1)

,blockHeight,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_feeTransfer,transactions_userCommands,0,fee,recipient,type
0,1,0,3NKeMoncuHab5ScarV5ViyF16cJPT4taWNSaTLS64Dp67w...,0,720000000000,B62qiy32p8kAKnny8ZFwoMhYpBppM1DWVCqAPBYNcXnsAH...,NaN,[],NaN,NaN,NaN,NaN
1,2,0,3NLyWnjZqUECniE1q719CoLmes6WDQAod4vrTeLfN7XXJb...,0,720000000000,B62qqa9g4CFfkSuX2j22S52z6UfcDcS9tMTgQrFKZ21v7G...,NaN,[],NaN,NaN,NaN,NaN
2,3,0,3NKd5So3VNqGZtRZiWsti4yaEe1fX79yz5TbfG6jBZqgMn...,120000000,720000000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,"{'fee': '120000000', 'recipient': 'B62qqSUUCno...","[{'amount': 10000, 'blockHeight': 3, 'blockSta...",NaN,120000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,Fee_transfer
3,4,0,3NL9qBsNibXPm5Nh8cSg5CCqrbzX5VUVY9gJzAbg7EVCF3...,60000000,720000000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,"{'fee': '60000000', 'recipient': 'B62qjZXMHQij...","[{'amount': 10000, 'blockHeight': 4, 'blockSta...",NaN,60000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,Fee_transfer
4,5,0,3NKQUoBfi9vkbuqtDJmSEYBQrcSo4GjwG8bPCiii4yqM8A...,70000000,720000000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,"{'fee': '70000000', 'recipient': 'B62qkiJuTwdJ...","[{'amount': 1000, 'blockHeight': 5, 'blockStat...",NaN,70000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,Fee_transfer
...,...,...,...,...,...,...,...,...,...,...,...,...
95,60,0,3NKu5MoEy2xvRMpKCg5FYVJ13paiFPzM3cuof5qr46v7Xf...,10000000,720000000000,B62qoigHEtJCoZ5ekbGHWyr9hYfc6fkZ2A41h9vvVZuvty...,"{'fee': '10000000', 'recipient': 'B62qoigHEtJC...","[{'amount': 1000, 'blockHeight': 60, 'blockSta...",NaN,10000000,B62qoigHEtJCoZ5ekbGHWyr9hYfc6fkZ2A41h9vvVZuvty...,Fee_transfer
96,60,0,3NKCdQt2pAeLqrPYxLHNs7UeusGGrTL8X6HcaFs8yJH4YA...,10000000,720000000000,B62qmvHQzJmT2rKE1F9RemenGRG8BfXT1Kurve3eT4iC2H...,"{'fee': '10000000', 'recipient': 'B62qmvHQzJmT...","[{'amount': 1000, 'blockHeight': 60, 'blockSta...",NaN,10000000,B62qmvHQzJmT2rKE1F9RemenGRG8BfXT1Kurve3eT4iC2H...,Fee_transfer
97,60,0,3NKYioZFVfvLw6BRmfnbpQRKaLcLQzyo6YTZ5Gr2TPtY4R...,10000000,720000000000,B62qmMkbajiY3bdVRjv5bx3yxjp5sBHHQuu8M11sDdjtBy...,"{'fee': '10000000', 'recipient': 'B62qmMkbajiY...","[{'amount': 1000, 'blockHeight': 60, 'blockSta...",NaN,10000000,B62qmMkbajiY3bdVRjv5bx3yxjp5sBHHQuu8M11sDdjtBy...,Fee_transfer
98,61,0,3NL6SMuZKjUrWYLoayQrRmsFbFzwfAp7WZydHkXaGakTzN...,10000000,720000000000,B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjC...,"{'fee': '10000000', 'recipient': 'B62qjBsp2zWg...","[{'amount': 1000, 'blockHeight': 61, 'blockSta...",NaN,10000000,B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjC...,Fee_transfer


In [191]:
#final_exploded = tmp_fees_df.explode(column='transactions_feeTransfer')

In [192]:
#final_exploded.groupby('transactions_coinbaseReceiverAccount_publicKey')

In [196]:
tmp_fees_df.groupby('transactions_coinbaseReceiverAccount_publicKey')

,blockHeight,snarkFees,stateHash,txFees,transactions_coinbase,transactions_coinbaseReceiverAccount_publicKey,transactions_feeTransfer,transactions_userCommands
0,1,0,3NKeMoncuHab5ScarV5ViyF16cJPT4taWNSaTLS64Dp67w...,0,720000000000,B62qiy32p8kAKnny8ZFwoMhYpBppM1DWVCqAPBYNcXnsAH...,NaN,[]
1,2,0,3NLyWnjZqUECniE1q719CoLmes6WDQAod4vrTeLfN7XXJb...,0,720000000000,B62qqa9g4CFfkSuX2j22S52z6UfcDcS9tMTgQrFKZ21v7G...,NaN,[]
2,3,0,3NKd5So3VNqGZtRZiWsti4yaEe1fX79yz5TbfG6jBZqgMn...,120000000,720000000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,"{'fee': '120000000', 'recipient': 'B62qqSUUCno...","[{'amount': 10000, 'blockHeight': 3, 'blockSta..."
3,4,0,3NL9qBsNibXPm5Nh8cSg5CCqrbzX5VUVY9gJzAbg7EVCF3...,60000000,720000000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,"{'fee': '60000000', 'recipient': 'B62qjZXMHQij...","[{'amount': 10000, 'blockHeight': 4, 'blockSta..."
4,5,0,3NKQUoBfi9vkbuqtDJmSEYBQrcSo4GjwG8bPCiii4yqM8A...,70000000,720000000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,"{'fee': '70000000', 'recipient': 'B62qkiJuTwdJ...","[{'amount': 1000, 'blockHeight': 5, 'blockStat..."
...,...,...,...,...,...,...,...,...
95,60,0,3NKu5MoEy2xvRMpKCg5FYVJ13paiFPzM3cuof5qr46v7Xf...,10000000,720000000000,B62qoigHEtJCoZ5ekbGHWyr9hYfc6fkZ2A41h9vvVZuvty...,"{'fee': '10000000', 'recipient': 'B62qoigHEtJC...","[{'amount': 1000, 'blockHeight': 60, 'blockSta..."
96,60,0,3NKCdQt2pAeLqrPYxLHNs7UeusGGrTL8X6HcaFs8yJH4YA...,10000000,720000000000,B62qmvHQzJmT2rKE1F9RemenGRG8BfXT1Kurve3eT4iC2H...,"{'fee': '10000000', 'recipient': 'B62qmvHQzJmT...","[{'amount': 1000, 'blockHeight': 60, 'blockSta..."
97,60,0,3NKYioZFVfvLw6BRmfnbpQRKaLcLQzyo6YTZ5Gr2TPtY4R...,10000000,720000000000,B62qmMkbajiY3bdVRjv5bx3yxjp5sBHHQuu8M11sDdjtBy...,"{'fee': '10000000', 'recipient': 'B62qmMkbajiY...","[{'amount': 1000, 'blockHeight': 60, 'blockSta..."
98,61,0,3NL6SMuZKjUrWYLoayQrRmsFbFzwfAp7WZydHkXaGakTzN...,10000000,720000000000,B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjC...,"{'fee': '10000000', 'recipient': 'B62qjBsp2zWg...","[{'amount': 1000, 'blockHeight': 61, 'blockSta..."


In [197]:
tmp_fees_df

(100, 8)

In [193]:
#tmp_fees_df.groupby('transactions_coinbaseReceiverAccount_publicKey')

In [166]:
fees_df.fee.fillna(0, inplace=True)
fees_df.fee = fees_df.fee.astype(int)

In [167]:
total_fee_transfers = fees_df[fees_df['type'] == 'Fee_transfer'].fee.sum()
total_fee_transfers

1490000000

In [168]:
fee_transfers_by_coinbase = fees_df[fees_df['type'] == 'Fee_transfer_via_coinbase'].fee.sum()
fee_transfers_by_coinbase

0

In [145]:
tmp_df = df.transactions_feeTransfer.explode()
exploded_fees = pd.DataFrame(tmp_df).transactions_feeTransfer.apply(pd.Series)

In [142]:
exploded_fees

,0,fee,recipient,type
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,120000000,B62qqSUUCnoC8Vehw5xwhrnaNxhk6Xe3FcBhngoxyXCbJB...,Fee_transfer
3,NaN,60000000,B62qjZXMHQijyxLPJHNk4QvXwYqgTj3V7X59o82AaD8Tqj...,Fee_transfer
4,NaN,70000000,B62qkiJuTwdJBARAPGAvStuEa37kZVZPyDrQoUCuM7WQUm...,Fee_transfer
...,...,...,...,...
95,NaN,10000000,B62qoigHEtJCoZ5ekbGHWyr9hYfc6fkZ2A41h9vvVZuvty...,Fee_transfer
96,NaN,10000000,B62qmvHQzJmT2rKE1F9RemenGRG8BfXT1Kurve3eT4iC2H...,Fee_transfer
97,NaN,10000000,B62qmMkbajiY3bdVRjv5bx3yxjp5sBHHQuu8M11sDdjtBy...,Fee_transfer
98,NaN,10000000,B62qjBsp2zWgyrbaSqVMcduz7A5s1sT63epFUVGFRjCfjC...,Fee_transfer


In [139]:
pd.melt(pd.DataFrame(k), var_name='transactions_feeTransfer')

,transactions_feeTransfer,value
0,transactions_feeTransfer,NaN
1,transactions_feeTransfer,NaN
2,transactions_feeTransfer,"{'fee': '120000000', 'recipient': 'B62qqSUUCno..."
3,transactions_feeTransfer,"{'fee': '60000000', 'recipient': 'B62qjZXMHQij..."
4,transactions_feeTransfer,"{'fee': '70000000', 'recipient': 'B62qkiJuTwdJ..."
...,...,...
95,transactions_feeTransfer,"{'fee': '10000000', 'recipient': 'B62qoigHEtJC..."
96,transactions_feeTransfer,"{'fee': '10000000', 'recipient': 'B62qmvHQzJmT..."
97,transactions_feeTransfer,"{'fee': '10000000', 'recipient': 'B62qmMkbajiY..."
98,transactions_feeTransfer,"{'fee': '10000000', 'recipient': 'B62qjBsp2zWg..."
